In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [12]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_5classes__Goertzel.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [13]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33
0,0.096944,0.108457,0.105318,0.084798,0.061510,0.038266,0.025620,0.017342,0.012191,0.007411,...,0.000859,0.000701,0.000510,0.000584,0.000748,0.000531,0.000930,0.000709,0.000391,0
1,0.138950,0.141066,0.127325,0.091396,0.051916,0.026497,0.012712,0.007062,0.004824,0.002380,...,0.000551,0.000582,0.000436,0.001493,0.000474,0.001178,0.000666,0.000391,0.001563,0
2,0.110719,0.120428,0.114312,0.099107,0.070298,0.049885,0.031301,0.024168,0.015279,0.014042,...,0.000919,0.000514,0.000456,0.000633,0.000753,0.000268,0.000705,0.000911,0.001172,0
3,0.106398,0.123849,0.115379,0.090015,0.063167,0.044005,0.030299,0.021928,0.018722,0.008435,...,0.000312,0.001194,0.000493,0.000117,0.000508,0.000501,0.000129,0.001024,0.000859,0
4,0.094950,0.104976,0.102646,0.089088,0.066788,0.045090,0.028919,0.019888,0.011982,0.009983,...,0.000881,0.000437,0.000709,0.000532,0.000773,0.000179,0.000886,0.000564,0.000312,0


In [14]:
dataset.shape

(49668, 33)

# Data preprocessing

In this step we are going standardize our dataset. The pipeline.pkl will be loaded, and it contains the weights of normalization onf DS1 set.

In [15]:
from sklearn.externals import joblib

pipeline = joblib.load('./Models/pipeline.pkl')
pipeline_part = pipeline.steps[0][1]

Separete features from labels.

In [16]:
data_std = pipeline_part.transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Load classifiers

In [17]:
from sklearn.externals import joblib

random_search = joblib.load('./Models/classifiers.pkl')

In [22]:
from sklearn.naive_bayes import GaussianNB

bayes_clf = GaussianNB()
bayes_clf.fit(data_std, data_label)

GaussianNB(priors=None)

In [23]:
svm_clf = random_search['SVM-RBF'].best_estimator_
mlp_clf = random_search['MLP'].best_estimator_

classifiers = {'MLP-238': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# **TEST** on DS2 set

In [24]:
clf_outputs = {'true': dict((k, {}) for k in classifiers.keys()), 
               'pred': dict((k, {}) for k in classifiers.keys())}
results = dict((k, {}) for k in classifiers.keys())

for clf in classifiers.keys():
    print(clf)
    clf_outputs['true'][clf][0] = data_label
    clf_outputs['pred'][clf][0] = classifiers[clf].predict(data_std)
    

results = results_clf(5, clf_outputs['true'], clf_outputs['pred'])

MLP-238
SVM-RBF
Naive-Bayes


/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Goertzel/Experiment_06_scikitLearn__allCLF_onDS2_allFeatures/experiment.py:152: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,3] = (2*metrics_class[i,2]*metrics_class[i,1])/(metrics_class[i,2] + metrics_class[i,1])
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Goertzel/Experiment_06_scikitLearn__allCLF_onDS2_allFeatures/experiment.py:150:

# Save results to CSVs and figures

In [26]:
export_results(results, 'validation')

MLP-238
SVM-RBF
Naive-Bayes


/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Goertzel/Experiment_06_scikitLearn__allCLF_onDS2_allFeatures/experiment.py:286: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)
